Dec. 13, Tianjin <br>
Project begins

Dec. 19

__Yuqing DeepThinking: <font color = "DarkRed"> Note that for the examples in hands-on, `train_test_split` is a necessary procedure. Which I think is what the actual case looks like: to follow industry standard, from the available dataset we get train, validation and test sets.</font>__ 

__Yuqing DeepThinking: <font color = "DarkRed"> However, for a Kaggle competition, train and test set are always provided.</font>__  <br>
__<font color = "DarkRed"> Does that mean for Kaggle projects, the test set is ready and we need to split validation set from train? I believe that is the case. If `test` is truly the validation set, then it should include label (`SalePrice`), which it does not. </font>__

__In sum, CV procedures should be considered in model fitting.__ <br>

### <font color = "red"> Better Evaluation Using Cross-validation </font>

realized by `from sklearn.model_selection import cross_val_score  train_test_split  RandomizedSearchCV`

Dec. 21  Morning

Fit `lin_svm` linear SVM to train the `train_prepared`; compute RMSE implementing CV. Result shows that it works, meaning, without error on the prediction task on this housing dataset, but with poor results. 

In [3]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


##### above: Heron end-to-end
##### below: Kaggle Top 4%

import pandas as pd

#import seaborn as sns
#color = sns.color_palette()
#sns.set_style('darkgrid')

from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points



In [4]:
import pandas as pd

train = pd.read_csv ('train.csv')
test = pd.read_csv ('test.csv')
test_ID = test['Id']
# print (df)
train.head(7)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.000,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.000,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000


In [5]:
#train.info()

Pay attention to the non-null attibutes for each attribute and dtypes: float64(3), int64(34), object(43)!

In [6]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000,1460.000,1201.000,1460.000,1460.000,1460.000,1460.000,1460.000,1452.000,1460.000,...,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000,1460.000
mean,730.500,56.897,70.050,10516.828,6.099,5.575,1971.268,1984.866,103.685,443.640,...,94.245,46.660,21.954,3.410,15.061,2.759,43.489,6.322,2007.816,180921.196
std,421.610,42.301,24.285,9981.265,1.383,1.113,30.203,20.645,181.066,456.098,...,125.339,66.256,61.119,29.317,55.757,40.177,496.123,2.704,1.328,79442.503
min,1.000,20.000,21.000,1300.000,1.000,1.000,1872.000,1950.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,2006.000,34900.000
25%,365.750,20.000,59.000,7553.500,5.000,5.000,1954.000,1967.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,2007.000,129975.000
50%,730.500,50.000,69.000,9478.500,6.000,5.000,1973.000,1994.000,0.000,383.500,...,0.000,25.000,0.000,0.000,0.000,0.000,0.000,6.000,2008.000,163000.000
75%,1095.250,70.000,80.000,11601.500,7.000,6.000,2000.000,2004.000,166.000,712.250,...,168.000,68.000,0.000,0.000,0.000,0.000,0.000,8.000,2009.000,214000.000
max,1460.000,190.000,313.000,215245.000,10.000,9.000,2010.000,2010.000,1600.000,5644.000,...,857.000,547.000,552.000,508.000,480.000,738.000,15500.000,12.000,2010.000,755000.000


38 numeric (including categorical ones coded as 0,1,2..)

__!!!!Kaggle already assigned test set, so stratified sampling for test set unnecessary__

### Looking for Correlations

Start with hands-on startegy, limited compared to Kagglers' work.

In [7]:
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the 'Id' colum since it's unnecessary for the prediction process.
train.drop("Id", axis = 1, inplace = True) # inplace = True: original object is changed
test.drop("Id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))

The train data size before dropping Id feature is : (1460, 81) 
The test data size before dropping Id feature is : (1459, 80) 

The train data size after dropping Id feature is : (1460, 80) 
The test data size after dropping Id feature is : (1459, 79) 


In [8]:
corr_matrix = train.corr()

In [9]:
#corr_matrix

In [10]:
#corr_matrix["SalePrice"].sort_values(ascending = False)

In [11]:
from pandas.plotting import scatter_matrix

attributes = ["SalePrice", "OverallQual", "GrLivArea",
              "GarageCars"]
#scatter_matrix(train[attributes], figsize=(12, 8))


### Experimenting with Attribute Combinations

In [12]:
## Prepare the Data for Machine Learning Algorithms p.62

In [13]:
print(train.shape)
print(test.shape)

(1460, 80)
(1459, 79)


__save `SalesPrice` / label for train : `train_labels` for future use__

In [14]:
train_labels = train['SalePrice']#.copy()

__create all_data__

In [15]:
train_noSale = train.drop(['SalePrice'], axis = 1, inplace = False)

# to create all_data, drop train y_label
all_data = pd.concat((train_noSale, test))


In [16]:
# TRAIN STILL has SalePrice
# train

In [17]:
all_data.shape

(2919, 79)

In [18]:
sample_incomplete_rows = train[train.isnull().any(axis=1)].head()
sample_incomplete_rows

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## `NA.(all_data)`

### Examining --> Clean and Impute. `.fillna("None")`

- Overview of the NA situation

- missing ratio

In [19]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,99.657
MiscFeature,96.403
Alley,93.217
Fence,80.439
FireplaceQu,48.647
LotFrontage,16.650
GarageFinish,5.447
GarageYrBlt,5.447
GarageQual,5.447
GarageCond,5.447


`.fillna` for both numerical categorical ones 

Exactly following Stacked Regression code trunk [14] - [33]
https://www.kaggle.com/yuqingxue/stacked-regressions-top-4-on-leaderboard

- **PoolQC** : data description says NA means "No  Pool". That make sense, given the huge ratio of missing value (+99%) and majority of houses have no Pool at all in general. 

In [20]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")

- **MiscFeature** : data description says NA means "no misc feature"


In [21]:
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")

- **Alley** : data description says NA means "no alley access"

In [22]:
all_data["Alley"] = all_data["Alley"].fillna("None")

- **Fence** : data description says NA means "no fence"

In [23]:
all_data["Fence"] = all_data["Fence"].fillna("None")

- **FireplaceQu** : data description says NA means "no fireplace"

In [24]:
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

- **LotFrontage** : Since the area of each street connected to the house property most likely have a similar area to other houses in its neighborhood , we can **fill in missing values by the median LotFrontage of the neighborhood**.

In [25]:
### Yuqing:Very nice thinking!!!
# Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood

all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

- **GarageType, GarageFinish, GarageQual and GarageCond** : Replacing missing data with None

In [26]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

- **GarageYrBlt, GarageArea and GarageCars** : Replacing missing data with 0 (Since No garage = no cars in such garage.)


In [27]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

- **BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath** : missing values are likely zero for having no basement

In [28]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

- **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2** : For all these categorical basement-related features, NaN means that there is no  basement.

In [29]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

- **MasVnrArea and MasVnrType** : NA most likely means no masonry veneer for these houses. We can fill 0 for the area and None for the type. 

In [30]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

- **MSZoning (The general zoning classification)** :  'RL' is by far  the most common value.  So we can fill in missing values with 'RL'

In [31]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

- **Utilities** : For this categorical feature all records are "AllPub", except for one "NoSeWa"  and 2 NA . Since the house with 'NoSewa' is in the training set, **this feature won't help in predictive modelling**. We can then safely  remove it.

In [32]:
all_data = all_data.drop(['Utilities'], axis=1)

- **Functional** : data description says NA means typical

In [33]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

- **Electrical** : It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.


In [34]:
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

- **KitchenQual**: Only one NA value, and same as Electrical, we set 'TA' (which is the most frequent)  for the missing value in KitchenQual.

In [35]:
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

- **Exterior1st and Exterior2nd** : Again Both Exterior 1 & 2 have only one missing value. We will just substitute in the most common string

In [36]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

- **SaleType** : Fill in again with most frequent which is "WD"

In [37]:
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

- **MSSubClass** : Na most likely means No building class. We can replace missing values with None


In [38]:
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

__Check: Is there any remaining missing value ?__

In [39]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


NA HANDLED.

## Feature Engineering: 

### Transforming some numerical variables that are really categorical

In [40]:
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
### Yuqing Why?
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

__`all_data` cleaned, generate train_num and train_cat from all_data__

In [41]:
#all_data.info()  # float64: BsmtFullBath, BsmtHalfBath

`train_num`

In [42]:
ntrain = train.shape[0]
ntrain

1460

In [43]:
ntest = test.shape[0]
ntest

1459

### Attributes adder

<font color = darkorange> My attempt pre-enlightment in the Kithchen and Tim Hortons mid November </font>

Yuqing: Attempt currently failed when try to include teh following in num.pipeline ,

So for now just do it manually, which means will do it manually in `test_clean`

__Author's move:__

In [44]:
# Adding total sqfootage feature 
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

__Yuqing's working move:__

In [45]:
all_data["TotalBath"] = all_data["BsmtFullBath"] + (0.5 * all_data["BsmtHalfBath"]) + \
all_data["FullBath"] + (0.5 * all_data["HalfBath"])

In [46]:
all_data["TotalSF_s2"] = all_data["TotalSF"] ** 2  
all_data["TotalSF_s3"] = all_data["TotalSF"] ** 3
all_data["TotalSF_sq"] = np.sqrt(all_data["TotalSF"])

In [53]:
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF,TotalBath,TotalSF_s2,TotalSF_s3,TotalSF_sq
0,60,RL,5.831,19.212,Pave,None,Reg,Lvl,Inside,Gtl,...,0.000,2,2008,WD,Normal,14.977,1.687,63.590,221.407,5.380
1,20,RL,6.221,19.712,Pave,None,Reg,Lvl,FR2,Gtl,...,0.000,5,2007,WD,Normal,14.923,1.378,63.243,219.719,5.366
2,60,RL,5.915,20.347,Pave,None,IR1,Lvl,Inside,Gtl,...,0.000,9,2008,WD,Normal,15.150,1.687,64.718,226.924,5.427
3,70,RL,5.685,19.692,Pave,None,IR1,Lvl,Corner,Gtl,...,0.000,2,2006,WD,Abnorml,14.857,1.194,62.816,217.649,5.348
4,60,RL,6.315,21.325,Pave,None,IR1,Lvl,FR2,Gtl,...,0.000,12,2008,WD,Normal,15.852,1.687,69.392,250.237,5.617


### Deal With Skewed Features for all_data

In [48]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index  # numeric_feats is index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
TotalSF_s3,24.199
MiscVal,21.947
PoolArea,16.898
LotArea,12.822
LowQualFinSF,12.089
3SsnPorch,11.376
TotalSF_s2,8.008
KitchenAbvGr,4.302
BsmtFinSF2,4.146
EnclosedPorch,4.004


In [49]:
skewness = skewness[abs(skewness) > 0.75] # why 0.75?
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
    
#all_data[skewed_features] = np.log1p(all_data[skewed_features])

There are 37 skewed numerical features to Box Cox transform


In [54]:
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF,TotalBath,TotalSF_s2,TotalSF_s3,TotalSF_sq
0,60,RL,5.831,19.212,Pave,None,Reg,Lvl,Inside,Gtl,...,0.000,2,2008,WD,Normal,14.977,1.687,63.590,221.407,5.380
1,20,RL,6.221,19.712,Pave,None,Reg,Lvl,FR2,Gtl,...,0.000,5,2007,WD,Normal,14.923,1.378,63.243,219.719,5.366
2,60,RL,5.915,20.347,Pave,None,IR1,Lvl,Inside,Gtl,...,0.000,9,2008,WD,Normal,15.150,1.687,64.718,226.924,5.427
3,70,RL,5.685,19.692,Pave,None,IR1,Lvl,Corner,Gtl,...,0.000,2,2006,WD,Abnorml,14.857,1.194,62.816,217.649,5.348
4,60,RL,6.315,21.325,Pave,None,IR1,Lvl,FR2,Gtl,...,0.000,12,2008,WD,Normal,15.852,1.687,69.392,250.237,5.617


###  NA handled, attributes combined/added, skewness adjuested

### Now, scale for numeric attributes (  `StandardScaler()`) 

#### First, create sub-dataset: all_data_num

In [ ]:
all_data.info()

float 37, object 46

__subset by column data type: <font color = "Teal"> .select_dtypes(exclude/include='object') </font>__

In [60]:
all_data_num = all_data.select_dtypes(exclude='object')

In [ ]:
all_data_num.info()

###  num_pipeline:  first part of the transformation pipeline that impose on the numeric portions of the whole dataset (and will be on the test set)

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([      
        ('std_scaler', StandardScaler()),
    ])

all_data_num_scaled = num_pipeline.fit_transform(all_data_num)

In [64]:
all_data_num_scaled  # pipeline output numpy array

array([[-0.07803651, -0.14235981,  0.6808008 , ...,  0.13146232,
         0.10840405,  0.16375053],
       [ 0.56637451,  0.11041515,  0.02343572, ...,  0.07768212,
         0.05446504,  0.11065539],
       [ 0.0601593 ,  0.43144318,  0.6808008 , ...,  0.30639689,
         0.28477591,  0.33515654],
       ...,
       [ 2.87840983,  1.66230964, -0.71924254, ..., -0.02124868,
        -0.0444085 ,  0.01248555],
       [-0.22167891,  0.2794391 , -0.71924254, ..., -0.83562745,
        -0.84084944, -0.82148998],
       [ 0.32188236,  0.11603365,  0.6808008 , ...,  0.64953711,
         0.63479119,  0.66576097]])

[Branch out: clearer & verification] make `all_data_num_scaled` a data frame

In [65]:
all_data_num_scaled = pd.DataFrame(
    all_data_num_scaled,
    columns=list(all_data_num.columns),
    index=all_data_num.index)

all_data_num_scaled.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,TotalSF,TotalBath,TotalSF_s2,TotalSF_s3,TotalSF_sq
0,-0.078,-0.142,0.681,1.041,0.896,1.179,0.800,-0.358,-0.546,-0.071,...,-0.424,-0.112,-0.308,-0.066,-0.186,0.154,1.463,0.131,0.108,0.164
1,0.566,0.110,0.023,0.161,-0.391,-0.779,0.974,-0.358,-0.131,0.421,...,-0.424,-0.112,-0.308,-0.066,-0.186,0.101,0.440,0.078,0.054,0.111
2,0.060,0.431,0.681,0.977,0.848,1.079,0.611,-0.358,0.168,0.018,...,-0.424,-0.112,-0.308,-0.066,-0.186,0.327,1.463,0.306,0.285,0.335
3,-0.321,0.100,0.681,-1.868,-0.679,-0.779,0.235,-0.358,0.330,-0.223,...,2.904,-0.112,-0.308,-0.066,-0.186,0.035,-0.169,0.012,-0.012,0.045
4,0.721,0.926,1.272,0.944,0.754,1.503,0.761,-0.358,0.257,0.295,...,-0.424,-0.112,-0.308,-0.066,-0.186,1.027,1.463,1.031,1.030,1.025


`StandardScaler()` impose on `all_data_num` seems fine!

[Return to main]

### OneHot (dummy) for cat

In [67]:
all_data_cat = all_data.select_dtypes(include='object')

In [68]:
all_data_cat.head()

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,Pave,None,Reg,Lvl,Inside,Gtl,CollgCr,Norm,...,TA,TA,Y,None,None,None,2,2008,WD,Normal
1,20,RL,Pave,None,Reg,Lvl,FR2,Gtl,Veenker,Feedr,...,TA,TA,Y,None,None,None,5,2007,WD,Normal
2,60,RL,Pave,None,IR1,Lvl,Inside,Gtl,CollgCr,Norm,...,TA,TA,Y,None,None,None,9,2008,WD,Normal
3,70,RL,Pave,None,IR1,Lvl,Corner,Gtl,Crawfor,Norm,...,TA,TA,Y,None,None,None,2,2006,WD,Abnorml
4,60,RL,Pave,None,IR1,Lvl,FR2,Gtl,NoRidge,Norm,...,TA,TA,Y,None,None,None,12,2008,WD,Normal


###  cat_pipeline:  second part of the transformation pipeline that impose on the categorical portions of the whole dataset (and will be on the test set)

#### Yuqing: for now, it is just `OneHotEncoder()`

In [69]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
all_data_cat_1hot = cat_encoder.fit_transform(all_data_cat)
print(all_data_cat_1hot.shape)
all_data_cat_1hot

(2919, 306)


<2919x306 sparse matrix of type '<class 'numpy.float64'>'
	with 134274 stored elements in Compressed Sparse Row format>

## !! Encapsulate in `full_pipeline` to be imposed on `all_data`!! 

### Yuqing: Important Reminder:  `imputer` and `add_attributes` currently are implemented manually, on `all_data`, not only on num 

In [72]:
from sklearn.compose import ColumnTransformer

num_attribs = list(all_data_num)
cat_attribs = list(all_data_cat)

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs), # for now, just StandardScaler()
        ("cat", OneHotEncoder(), cat_attribs), # not even a pipeline like in the book
    ])



Noted: `full_pipeline`, as put it, implemented on full dataset `all_data`

In [73]:
all_data_prepared = full_pipeline.fit_transform(all_data) 

In [74]:
all_data_prepared # again, also an array

<2919x343 sparse matrix of type '<class 'numpy.float64'>'
	with 242277 stored elements in Compressed Sparse Row format>

`all_data` --> `all_data_prepared`

### Feature Engineering Concluded!

# Select and Train a Model

Don't forget: `all_data` includes train and test from Kaggle that we need to split validation set from train. In fact,

In [79]:
print(train.shape[0]) # orginal train: 1460 instances
all_data_prepared.shape # all_data got 2919 instances

1460


(2919, 343)

__So, need to get the current train.__ We call it `train_prepared`

In [80]:
train_prepared = all_data_prepared[:train.shape[0]]

In [81]:
train_prepared.shape

(1460, 343)

In [120]:
train_labels

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

The rest in `all_data_prepared` is `test_prepared` 

In [87]:
test_prepared = all_data_prepared[train.shape[0]:]

In [88]:
test_prepared.shape

(1459, 343)

### Trivial: linear regression train performance pre-CV

In [84]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_prepared, train_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [89]:
from sklearn.metrics import mean_squared_error
train_predictions = lin_reg.predict(train_prepared)
lin_mse = mean_squared_error(train_labels, train_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

20517.4808942295

In [94]:
lin_reg.score(train_prepared, train_labels) # R^2

0.9332517625177763

### Trivial: Decision Tree train performance pre-CV

In [91]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_prepared, train_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [92]:
train_predictions = tree_reg.predict(train_prepared)
tree_mse = mean_squared_error(train_labels, train_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [95]:
tree_reg.score(train_prepared, train_predictions) # R^2, best score for overfitting

1.0

### Promising: RF train performance pre-CV

In [ ]:
import time
start_time = time.time()


from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(train_prepared, train_labels)

print("--- %s seconds ---" % (time.time() - start_time))

train performance

In [124]:
train_predictions = forest_reg.predict(train_prepared)
forest_mse = mean_squared_error(train_labels, train_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

11169.384114236733

### Trivial: linear SVM pre-CV :     

try svm in prediction problem


In [123]:
from sklearn.svm import LinearSVC

lin_svm = LinearSVC()
lin_svm.fit(train_prepared, train_labels)

/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [126]:
svm_train_predictions = lin_svm.predict(train_prepared)
lin_svm_mse = mean_squared_error(train_labels, svm_train_predictions)
lin_svm_rmse = np.sqrt(lin_svm_mse)
lin_svm_rmse

196.28397097133015

Overfitting probable

In [127]:
svm_train_predictions ## can make numerical prediction!!!!!!!

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

# Better Evaluation Using Cross-validation

Further confirm that `tree_reg` has serious overfitting issues 

## !!! ` cross_val_score` does `train_test_split`?

https://scikit-learn.org/stable/modules/cross_validation.html

__CV for decision tree `tree_reg` on train__

In [101]:
from sklearn.model_selection import cross_val_score

tree_scores = cross_val_score(tree_reg, train_prepared, train_labels,
                         scoring="neg_mean_squared_error", cv=10)
### Yuqing: score here is -MSE (a utility function)
tree_rmse_scores = np.sqrt(-tree_scores)

In [103]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)


Scores: [37821.8896876  36201.71087004 35805.45223778 40312.1441417
 38158.96873804 34879.12689365 41599.33027714 34854.90933238
 37165.04431826 40579.00602613]
Mean: 37737.75825227176
Standard deviation: 2295.653000322459


__mean: 37737.8, Standard deviation: 2295.7__

__CV for linear regression `lin_reg` on train__

In [104]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(lin_reg, train_prepared, train_labels,
                         scoring="neg_mean_squared_error", cv=10)
### Yuqing: lin_scores here is -MSE (a utility function)
lin_rmse_scores = np.sqrt(-lin_scores) # the score (standard error) we want

In [105]:
display_scores(lin_rmse_scores)

Scores: [24303.78131862 33829.02607801 24001.91174439 64623.8675378
 32113.76809777 60364.16297604 25137.85305732 23072.79792856
 75570.84546815 22137.70585227]
Mean: 38515.57200589125
Standard deviation: 19218.071142232875


__mean: 38515.6, Standard deviation: 19281__

__CV for Random Forest `forest_reg` on train__

In [110]:
import time
start_time = time.time()


from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, train_prepared, train_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)


print("--- %s seconds ---" % (time.time() - start_time))

Scores: [23806.73044767 25836.08296132 24907.7747697  38065.33362325
 34297.96691574 24016.31558961 21949.65709364 24050.42123512
 41567.391628   26120.97415927]
Mean: 28461.864842333212
Standard deviation: 6529.551723823159
--- 120.0295479297638 seconds ---


__mean: 28461.9, Standard deviation: 6529__

__CV for linear svm `lin_svm` on train__

In [128]:
import time
start_time = time.time()


from sklearn.model_selection import cross_val_score

lin_svm_scores = cross_val_score(lin_svm, train_prepared, train_labels,
                                scoring="neg_mean_squared_error", cv=10)
lin_svm_rmse_scores = np.sqrt(-lin_svm_scores)
display_scores(lin_svm_rmse_scores)


print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

Scores: [43267.45386681 48702.22642298 71677.94964799 48967.97034874
 47777.18866337 57999.17339291 70292.53208897 59188.94110979
 53041.95991647 61474.97420434]
Mean: 56239.03696623778
Standard deviation: 9136.92438847384

--- 23.932585954666138 seconds ---


/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


__mean: 56239.0, Standard deviation: 9137__

__so, `linear_svm` can perform prediction, just really badly__

__*Aggregate the previous evaluations scores:*__

In [115]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    print() # an empty line

In [129]:
from sklearn.model_selection import cross_val_score

for scores in (tree_rmse_scores, lin_rmse_scores, forest_rmse_scores, lin_svm_rmse_scores):
    display_scores(scores)

Scores: [37821.8896876  36201.71087004 35805.45223778 40312.1441417
 38158.96873804 34879.12689365 41599.33027714 34854.90933238
 37165.04431826 40579.00602613]
Mean: 37737.75825227176
Standard deviation: 2295.653000322459

Scores: [24303.78131862 33829.02607801 24001.91174439 64623.8675378
 32113.76809777 60364.16297604 25137.85305732 23072.79792856
 75570.84546815 22137.70585227]
Mean: 38515.57200589125
Standard deviation: 19218.071142232875

Scores: [23806.73044767 25836.08296132 24907.7747697  38065.33362325
 34297.96691574 24016.31558961 21949.65709364 24050.42123512
 41567.391628   26120.97415927]
Mean: 28461.864842333212
Standard deviation: 6529.551723823159

Scores: [43267.45386681 48702.22642298 71677.94964799 48967.97034874
 47777.18866337 57999.17339291 70292.53208897 59188.94110979
 53041.95991647 61474.97420434]
Mean: 56239.03696623778
Standard deviation: 9136.92438847384



Again, random forest much better, as anticipated.

### We got the RMSE of predictors on validation. Promising models proceed to be fine-tuned

which are uncorrelated? fine tuned then for stacking

## Fine-Tune Your Promising Model: random forest

### Grid Search

1. The first rough Grid Search __on random forest__

In [134]:
import time
start_time = time.time()


from sklearn.model_selection import GridSearchCV

param_grid = [
  # try 12 (3×4) combinations of hyperparameters
  {'n_estimators': [30, 100, 300], 'max_features': [10, 50, 100, 200, 300]},
  # then try 6 (2×3) combinations with bootstrap set as False
  {'bootstrap': [False], 'n_estimators': [30,100], 'max_features': [10, 100, 300]},
  ]

forest_reg = RandomForestRegressor(random_state=42) 
# train across 5 folds (train each model 5 times), that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,  ### forest_reg
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
### Yuqing: after that, grid_search saves the computed estimators, to be applied on the following
grid_search.fit(train_prepared, train_labels)


print("--- %s seconds ---" % (time.time() - start_time))

--- 612.4737260341644 seconds ---


__call attributes: `best_params_` , `best_estimator_` and `cv_results_`__

In [136]:
grid_search.best_params_  # 

{'bootstrap': False, 'max_features': 100, 'n_estimators': 100}

In [137]:
grid_search.best_estimator_ # current best model

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=100, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [138]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

31597.54394034685 {'max_features': 10, 'n_estimators': 30}
30569.623060759426 {'max_features': 10, 'n_estimators': 100}
30339.84513642285 {'max_features': 10, 'n_estimators': 300}
30585.29560575916 {'max_features': 50, 'n_estimators': 30}
30277.743317387532 {'max_features': 50, 'n_estimators': 100}
29487.868333481423 {'max_features': 50, 'n_estimators': 300}
30334.157675511622 {'max_features': 100, 'n_estimators': 30}
29629.479369602363 {'max_features': 100, 'n_estimators': 100}
29311.520868873933 {'max_features': 100, 'n_estimators': 300}
29786.30523291886 {'max_features': 200, 'n_estimators': 30}
29215.054259890727 {'max_features': 200, 'n_estimators': 100}
29459.75132341039 {'max_features': 200, 'n_estimators': 300}
29952.171582525178 {'max_features': 300, 'n_estimators': 30}
29592.281896782868 {'max_features': 300, 'n_estimators': 100}
29724.26903598751 {'max_features': 300, 'n_estimators': 300}
30450.94620728237 {'bootstrap': False, 'max_features': 10, 'n_estimators': 30}
29558.53

In [140]:
#pd.DataFrame(grid_search.cv_results_)

### Randomized Search

2. Try randomized search

From previous grid_search.best_params_: 'max_features': 100, 'n_estimators': 100

In [151]:
import time
start_time = time.time()


from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
  'n_estimators': randint(low=53, high=54),
  'max_features': randint(low=95, high=105), # by this way, only consider max_features = 99
}

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=15, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(train_prepared, train_labels)


print("--- %s seconds ---" % (time.time() - start_time))

--- 147.5145881175995 seconds ---


In [152]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

29167.06780605624 {'max_features': 101, 'n_estimators': 53}
28809.410105377934 {'max_features': 98, 'n_estimators': 53}
29831.6952357046 {'max_features': 102, 'n_estimators': 53}
28500.827174998678 {'max_features': 99, 'n_estimators': 53}
29167.06780605624 {'max_features': 101, 'n_estimators': 53}
29299.08997318461 {'max_features': 104, 'n_estimators': 53}
30046.592386150674 {'max_features': 97, 'n_estimators': 53}
29167.06780605624 {'max_features': 101, 'n_estimators': 53}
29831.6952357046 {'max_features': 102, 'n_estimators': 53}
28500.827174998678 {'max_features': 99, 'n_estimators': 53}
28809.410105377934 {'max_features': 98, 'n_estimators': 53}
29831.6952357046 {'max_features': 102, 'n_estimators': 53}
29831.6952357046 {'max_features': 102, 'n_estimators': 53}
30046.592386150674 {'max_features': 97, 'n_estimators': 53}
30029.84613004664 {'max_features': 100, 'n_estimators': 53}


Testimate: Important information best: 
28500.827174998678 {'max_features': 99, 'n_estimators': 53} Best!
      28510 {'max_features': 99, 'n_estimators': 54}
      28511: {'max_features': 99, 'n_estimators': 55}


__Yuqing: Based on the precious result, max_features: 99, n_estimator 53. <br>


__Yuqing: Eye-spotting best hyperparameters: 'max_features': 99, 'n_estimators': 53__

Which can be obtained by just call `rnd_search.best_estimator_`

In [166]:
rnd_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=99, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=53, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [171]:
final_randomForest_model = rnd_search.best_estimator_

In [177]:
my_model = final_randomForest_model

In [178]:
import joblib
joblib.dump(my_model, "randomForest_1221_1.pkl") # DIFF
#...
my_model_loaded = joblib.load("randomForest_1221_1.pkl") # DIFF

In [179]:
my_model_loaded

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=99, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=53, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

### RF fit on test

In [156]:
final_predictions = final_randomForest_model.predict(test_prepared)

### unlike hands_on: test is from Kaggle with 79 clumns (1 less than train ), so no SalePrice in test. so juest predict, don't create y_label. Kaggle has the y_label to score

In [157]:
final_predictions

array([124622.75, 158385.37, 178080.95, ..., 160426.46, 114661.5 ,
       216131.54])

In [158]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = final_predictions
sub.to_csv('submission_1221_1.csv',index=False)

### xgboost

In [180]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Yuqing: create validation set!!!

#################################################################################
xgb_train, xgb_val, xgb_train_labels, xgb_val_labels = train_test_split(train_prepared, train_labels, random_state=49) ########
#################################################################################

In [187]:
import xgboost
xgb_reg = xgboost.XGBRegressor(random_state=42)
xgb_reg.fit(xgb_train, xgb_train_labels)
y_pred = xgb_reg.predict(xgb_val)
val_error = mean_squared_error(xgb_val_labels, y_pred) # Not shown
print("Validation MSE:", np.sqrt(val_error) )  

/usr/local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:54:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Validation MSE: 22823.02774031364


In [ ]:
xgb_reg.fit(xgb_train, xgb_train_labels, eval_set=[(xgb_val, xgb_val_labels)], early_stopping_rounds=15)
y_pred = xgb_reg.predict(xgb_val)
val_error = mean_squared_error(xgb_val_labels, y_pred) # Not shown
print("Validation MSE:", np.sqrt(val_error) )  

This is just the performance of it on i validation set

## CV pending!

In [194]:
xgb_reg

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

it seems just the default parameters after fitting random spliited validtion set

In [195]:
xgb_withoutCV_prediction = xgb_reg.predict(test_prepared)

In [196]:
xgb_withoutCV_prediction

array([121591.2  , 159418.1  , 181481.12 , ..., 156509.4  , 121033.484,
       224477.22 ], dtype=float32)

In [197]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = xgb_withoutCV_prediction
sub.to_csv('submission_1221_2_preCV_XGB.csv',index=False)

### Result: 0.14159 on Kaggle, better than 0.14398 from the best random Forest

# Stacking!!!! follow chapter 7 exercise

__*Insert Stacked*__

In [124]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
#import lightgbm as lgb

Define a cross validation strategy

In [125]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state = 42).get_n_splits(train_prepared.values) ### author set random_state=42
    rmse= np.sqrt(-cross_val_score(model, train_prepred.values, train_labels, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [123]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [126]:
lasso

Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('lasso',
                 Lasso(alpha=0.0005, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=1, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

__Attempt: Write my own transformers for tasks suc as custom cleanuo operations or combining specific attributes__


__Do the rquivamant operation on copy all_data2 with the idea of pipeline with transformers!__

In [ ]:
#train_clean_num = train_clean.select_dtypes(exclude='object')

In [ ]:
#train_clean_num['TotalBsmtSF'] = train_clean_num['TotalBsmtSF'].astype(np.int64)
#train_clean_num['BsmtFullBath'] = train_clean_num['BsmtFullBath'].astype(np.int64)
#train_clean_num['BsmtHalfBath'] = train_clean_num['BsmtHalfBath'].astype(np.int64)

In [ ]:
#train_clean_num.columns.get_loc("TotalBsmtSF") # 9

In [ ]:
#train_clean_num.columns.get_loc("1stFlrSF") # 10

In [ ]:
#train_clean_num.columns.get_loc("2ndFlrSF") # 11

In [ ]:
#train_clean_num.columns.get_loc("BsmtFullBath") # 14

In [ ]:
#train_clean_num.columns.get_loc("BsmtHalfBath") # 15

In [ ]:
#train_clean_num.columns.get_loc("FullBath") # 16

In [ ]:
#train_clean_num.columns.get_loc("HalfBath") # 17

check titakSF int64 and float64

now check totalBath

In [ ]:
#from sklearn.base import BaseEstimator, TransformerMixin

# column index (seems necessary, X[:,"TotalBsmtSF"] not recognizable)
#ix_TotalBsmtSF, ix_1stFlrSF, ix_2ndFlrSF = 9,10,11

#ix_BsmtFullBath, ix_BsmtHalfBath, ix_FullBath, ix_HalfBath = 14,15,16,17

#class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    #def __init__(self, dataset): # no *args or **kargs
     #   self.dataset = dataset
    #def fit(self, X, y=None):
    #    return self  # nothing else to do
    #def transform(self, X):
     #   TotalSF = X[:,ix_TotalBsmtSF].astype(np.int64) + X[:,ix_1stFlrSF] + X[:,ix_2ndFlrSF]
     #   TotalSF = TotalSF.
     #   TotalBath = X[:,ix_BsmtFullBath].astype(np.int64) + X[:,ix_BsmtHalfBath].astype(np.int64) * 0.5 + X[:,ix_FullBath] +  X[:,ix_HalfBath] * 0.5      
      #  return np.c_[X, TotalSF, TotalBath]

Polynomial

The biggest problem: Poly is imposed on a dataset that TotalSF should be already created, which won;t be an oroginal one
    

Maybe impute, scale, combine attributes should be in pipe, but PolySF should be out!

In [ ]:
#from sklearn.base import BaseEstimator, TransformerMixin

#ix_TotalSF= 78

#class PolyAttributesAdder(BaseEstimator, TransformerMixin):
    #def __init__(self, dataset): # no *args or **kargs
        #self.dataset = dataset
    #def fit(self, X, y=None):
       # return self  # nothing else to do
   # def transform(self, X):
        #TotalSF_s2 = X[:,ix_TotalSF] **2
        #TotalSF_s3 = X[:,ix_TotalSF] **3
        #TotalSF_sq = X[:,ix_TotalSF] **(1/2) # np.sqrt won't work
        #return np.c_[X, TotalSF_s2, TotalSF_s3, TotalSF_sq]

In [ ]:
#attr_adder = PolyAttributesAdder(all_data2_combined)
#all_data2_combined_polyed = attr_adder.transform(all_data2_combined.values)

https://stackoverflow.com/questions/13021654/get-column-index-from-column-name-in-python-pandas

## Try pipe! compared to all_data see is.same

 combine: (pre poly): impose on dataset with both num and cat (ix index number)

In [ ]:

#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler

#combine_pipeline = Pipeline([
        #('imputer', NAimputer()),
        #('attribs_adder', CombinedAttributesAdder(train_clean)), # train_clean: num and cat
        #('scaler', StandardScaler()),
        
#    ])

#train_clean_combined = combine_pipeline.fit_transform(train_clean.to_numpy())